# Experimental code for calculating Ripleys K metrics

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


import pandas as pd
import os
# cell border experimental code: 

import pandas as pd
import numpy as np
from scipy.spatial import ConvexHull

In [2]:
df = pd.read_csv(r'combined_csvs\final_output.csv')
df

,POSITION X,POSITION Y,POSITION Z,UNIT,CATEGORY,COLLECTION,TIME,ID,UNNAMED: 8,CD11B,MDSC
0,64490.059,39696.629,-26,µm,Surface,Position,1,0,0,0,0
1,64524.977,41758.023,-19,µm,Surface,Position,1,1,0,0,0
2,64569.633,41724.699,-24,µm,Surface,Position,1,2,0,0,0
3,64520.539,41776.246,-19,µm,Surface,Position,1,3,0,0,0
4,64673.102,41623.758,-20,µm,Surface,Position,1,4,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
80344,62531.742,39202.859,-26,µm,Surface,Position,1,80344,0,0,0
80345,61932.430,39619.660,-21,µm,Surface,Position,1,80345,0,1,0
80346,61911.309,39644.695,-19,µm,Surface,Position,1,80346,0,1,0
80347,61936.102,39632.754,-20,µm,Surface,Position,1,80347,0,1,0


In [4]:
pts = df[['POSITION X', 'POSITION Y']].values             # shape (N,2) array

# 2) Compute the convex hull
hull = ConvexHull(pts)

# hull.simplices is an (E,2) array of index pairs (i, j)
# each representing an edge between pts[i] and pts[j].
edges = hull.simplices

# 3) Define a vectorized distance‐to‐segment function
def point_to_segment_distances(points, seg_start, seg_end):
    """
    points: (N,2) array
    seg_start, seg_end: each (2,) array
    returns: (N,) array of distances from each point to the segment
    """
    v = seg_end - seg_start                      # (2,)
    w = points - seg_start                       # (N,2)
    # projection factor of each point onto the infinite line
    t = np.einsum('ij,j->i', w, v) / np.dot(v, v)
    # clamp to [0,1] so we “snap” to the segment
    t_clamped = np.clip(t, 0.0, 1.0)             # (N,)
    proj = seg_start + t_clamped[:,None] * v     # (N,2)
    # euclidean distance from each point to its projection
    return np.linalg.norm(points - proj, axis=1)

# 4) Loop over edges, keep the minimum distance per point
all_dists = np.full(len(pts), np.inf)
for i, j in edges:
    start, end = pts[i], pts[j]
    d = point_to_segment_distances(pts, start, end)
    all_dists = np.minimum(all_dists, d)

# 5) Attach back to your DataFrame
df['dist_to_hull'] = all_dists

# 6) (Optional) Inspect
df

,POSITION X,POSITION Y,POSITION Z,UNIT,CATEGORY,COLLECTION,TIME,ID,UNNAMED: 8,CD11B,MDSC,dist_to_hull
0,64490.059,39696.629,-26,µm,Surface,Position,1,0,0,0,0,0.000000
1,64524.977,41758.023,-19,µm,Surface,Position,1,1,0,0,0,38.786774
2,64569.633,41724.699,-24,µm,Surface,Position,1,2,0,0,0,41.374558
3,64520.539,41776.246,-19,µm,Surface,Position,1,3,0,0,0,26.177811
4,64673.102,41623.758,-20,µm,Surface,Position,1,4,0,0,0,57.954712
...,...,...,...,...,...,...,...,...,...,...,...,...
80344,62531.742,39202.859,-26,µm,Surface,Position,1,80344,0,0,0,11.470460
80345,61932.430,39619.660,-21,µm,Surface,Position,1,80345,0,1,0,12.666644
80346,61911.309,39644.695,-19,µm,Surface,Position,1,80346,0,1,0,0.000000
80347,61936.102,39632.754,-20,µm,Surface,Position,1,80347,0,1,0,20.086775
